# Init Phase

In [1]:
import ABCD_ML
import numpy as np

Using TensorFlow backend.


In [2]:
data_dr = '/mnt/sdb2/ABCDFixRelease2p0p1/'

data1 = data_dr + 'abcd_smrip101.txt'
data2 = data_dr + 'abcd_smrip201.txt'

data_exp_dr = '/mnt/sdb2/2.0_ABCD_Data_Explorer/'
target = data_exp_dr + '2.0_NDA_Data/Other Non-Imaging/ABCD Longitudinal Tracking.csv'

name_map_loc = data_dr + 'Fix Release Notes 2.0.1_Public/24. ABCD_Release_2.0.1_Updates/abcd_2.0.1_mapping.csv'

In [ ]:
ML = ABCD_ML.ABCD_ML(exp_name='Site',
                     log_dr='',
                     existing_log='overwrite',
                     verbose=True,
                     notebook=True,
                     subject_id='src_subject_id',
                     eventname='baseline_year_1_arm_1',
                     use_default_subject_ids=True,
                     default_dataset_type='basic',
                     default_na_values=['777', '999'],
                     original_targets_key='targets',
                     low_memory_mode=False,
                     random_state=1)

In [ ]:
ML.Load_Name_Map(loc = name_map_loc,
                 dataset_type = 'explorer',
                 source_name_col = 'nda_name',
                 target_name_col = 'deap_name')

In [ ]:
ML.Load_Data(loc = [data1, data2],
             drop_keys = ['aseg_wm.hypointensities', 'smri_t2w'],
             drop_nan = False,
             filter_outlier_percent = None,
             winsorize_val = .001,
             unique_val_drop_thresh = 800,
             clear_existing = True)

In [ ]:
ML.Load_Targets(loc = target,
                col_name = 'site_id_l',
                data_type = 'c',
                dataset_type = 'explorer',
                categorical_drop_percent = None)

In [ ]:
ML.Show_Targets_Dist()

In [ ]:
ML.Load_Covars(loc = data1,
               col_names = ['sex', 'interview_age'],
               data_types = ['c', 'f'],
               drop_nan = False,
               clear_existing = True)

In [ ]:
ML.Show_Covars_Dist()

In [ ]:
# Stratify such that the target classes are evenly distributed between folds
ML.Define_Validation_Strategy(stratify=ML.original_targets_key)

In [ ]:
ML.Train_Test_Split(test_size=.2)

In [ ]:
#Regular_Iter_Imputer(imputer, inds) #float
#Regular_Iter_Imputer(imputer, inds) #binary
#Categorical_Iter_Imputer(imputer, encoder_inds, ordinal_inds, encoders) #categorical

In [ ]:
covar_scopes, cat_encoders = _get_covar_scopes()

In [ ]:
covar_scopes, cat_encoders

In [ ]:
ML.Set_Default_ML_Params(problem_type = 'categorical',
                         metric = ['macro f1', 'micro f1', 'macro roc auc'],
                         scaler = 'robust',
                         n_splits = 3,
                         n_repeats = 1,
                         n_jobs = 8)

In [ ]:
#ML.Set_ML_Verbosity(fold_name = True,
#                    time_per_fold = True,
#                    score_per_fold = True,
#                    fold_sizes = True,
#                    save_to_logs = False)

In [ ]:
raw_scores = ML.Evaluate(model_type = 'knn classifier',
                         metric = 'accuracy')

In [ ]:
raw_scores = ML.Evaluate(model_type = 'light gbm',
                         metric = ['micro f1', 'by class roc auc'])

In [ ]:
raw_scores = ML.Evaluate(model_type = 'elastic',
                         model_type_param_ind = 1,
                         search_type = 'random')

In [ ]:
ML.Evaluate(model_type = 'light gbm',
            model_type_param_ind = 1,
            search_type = 'random')